## Homework

In this homework, we will use Credit Card Data from [the previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/04-evaluation/homework.md).

## Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

### Solution
Run in your terminal:

1. pip install pipenv
2. pipenv install numpy pandas scikit-learn==1.0.2
3. pipenv --version
4. pipenv shell

In [1]:
!pipenv --version

pipenv, version 2020.11.15



## Question 2

* Use Pipenv to install Scikit-Learn version 1.0.2
* What's the first hash for scikit-learn you get in Pipfile.lock?

Answer: sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b

## Models

An already trained classification model will be used. The next step is to download the model and an associated DictVectorized to this task.
Download the following files.

In [2]:
PREFIX='https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework'

In [3]:

!python -m wget $PREFIX/model1.bin
!python -m wget $PREFIX/dv.bin


Saved under model1.bin

Saved under dv.bin


## Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

In [ ]:
import pickle

model_file = 'model1.bin'
dv_file = 'dv.bin'

with open(model_file,'rb') as f:
    model = pickle.load(f)
    
with open(dv_file,'rb') as f:
    dv = pickle.load(f)

In [6]:
client = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}

Customers can have a credit card `(1)` or not `(0)`.
The order of the prediction has the form:

[`Probability of not getting a credit card`, `Probability of getting a credit card`]

In [9]:
X = dv.transform([client])
y_pred = model.predict_proba(X)[0,1]
print('probability that this client will get a credit card ',y_pred)


probability that this client will get a credit card  0.16213414434326598


## Docker

Install [Docker](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/05-deployment/06-docker.md). We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.9.12-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.9.12-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

**1. Run in terminal**

`docker pull svizor/zoomcamp-model:3.9.12-slim`

## Question 5
So what's the size of this base image?

* 15 Mb
* 125 Mb
* 275 Mb
* 415 Mb

In [11]:
!docker images

REPOSITORY              TAG           IMAGE ID       CREATED      SIZE
svizor/zoomcamp-model   3.9.12-slim   571a6fdc554b   7 days ago   125MB
